# 7.7 稠密连接网络（DenseNet）
- **目录**
  - 7.7.1 从ResNet到DenseNet
  - 7.7.2 稠密块体
  - 7.7.3 过渡层
  - 7.7.4 DenseNet模型
  - 7.7.5 训练模型

- ResNet极大地改变了如何参数化深层网络中函数的观点。
- **稠密连接网络（DenseNet）** 在某种程度上是ResNet的逻辑扩展。

## 7.7.1 从ResNet到DenseNet

回想一下任意函数的**泰勒展开式（Taylor expansion）**，它把这个函数分解成越来越高阶的项。在$x$接近0时，

$$f(x) = f(0) + f'(0) x + \frac{f''(0)}{2!}  x^2 + \frac{f'''(0)}{3!}  x^3 + \ldots \tag{7.7.1}$$

同样，ResNet将函数展开为

$$f(\mathbf{x}) = \mathbf{x} + g(\mathbf{x})\tag{7.7.2}$$

也就是说，ResNet将$f$分解为两部分：**一个简单的线性项和一个复杂的非线性项**。

那么再向前拓展一步，如果我们想将$f$拓展成超过两部分的信息呢？
一种方案便是DenseNet。

<center>
    <img src="../img/densenet-block.svg" alt="ResNet（左）与 DenseNet（右）在跨层连接上的主要区别：使用相加和使用连结">
</center>
<center>图7.7.1 ResNet（左）与 DenseNet（右）在<b>跨层连接</b>上的主要区别：使用<b>相加</b>和使用<b>连结</b></center><br>

如 图7.7.1所示，ResNet和DenseNet的关键区别在于，DenseNet输出是**连接**（**用图中的$[,]$表示**）而不是如ResNet的简单相加。即将卷积层的输出与X在维度上连接起来形成新的张量，而不是做算术运算。
因此，在应用越来越复杂的函数序列后，我们执行从$\mathbf{x}$到其展开式的映射：

$$\mathbf{x} \to \left[
\mathbf{x},
f_1(\mathbf{x}),
f_2([\mathbf{x}, f_1(\mathbf{x})]), f_3([\mathbf{x}, f_1(\mathbf{x}), f_2([\mathbf{x}, f_1(\mathbf{x})])]), \ldots\right]\tag{7.7.3}$$

-------------
- **说明：泰勒展开式的论述与稠密网络有何联系？**
  - 泰勒展开式是一种将函数表示为无穷级数的方式，其中每一项表示函数在某一点的高阶导数乘以相应的幂次项。它可以看作是对函数的逐步逼近，从线性项开始，逐步加入更高阶的非线性项来提高近似的精度。
  - 在深度学习中，ResNet的残差块可以看作是对函数的一种**线性近似和修正**，即通过$f(\mathbf{x}) = \mathbf{x} + g(\mathbf{x})$的形式，将原始输入$\mathbf{x}$和一个非线性变换$g(\mathbf{x})$相加。这类似于泰勒展开中的线性部分加上一个修正项。
  - DenseNet则进一步扩展了这种思想。在DenseNet中，每一层的输出不仅是前一层的输出的非线性变换，还包括了所有之前层的输出的“连接”。这类似于将函数**展开为具有更多项的形式**，就像泰勒展开式中**更高阶项的累积**。具体来说，在DenseNet中，输出是通过连接（concatenation）之前所有层的输出来实现的：
  $$\mathbf{x} \to \left[
\mathbf{x},
f_1(\mathbf{x}),
f_2([\mathbf{x}, f_1(\mathbf{x})]), f_3([\mathbf{x}, f_1(\mathbf{x}), f_2([\mathbf{x}, f_1(\mathbf{x})])]), \ldots\right].
  $$

  - 这种连接方式使得每一层可以直接访问之前所有层的特征表示，从而能够更好地**复用和组合不同层的特征**。这种设计理念与泰勒展开式中的逐步累积更高阶项进行函数逼近的思路**相呼应**。在泰勒展开式中，增加更高阶的项可以提供**更精确的逼近**；在DenseNet中，通过连接所有之前层的输出，网络可以学习到**更丰富、更复杂的特征表示**。
  - 因此，泰勒展开式与DenseNet的联系在于：两者都**通过逐步增加信息（或者特征）来构建更复杂、精确的表示**。泰勒展开通过增加高阶导数项来逼近函数，而DenseNet通过连接不同层的输出来构建更复杂的特征空间。
------

最后，将这些展开式结合到多层感知机中，再次减少特征的数量。
实现起来非常简单：我们不需要添加术语，而是将它们连接起来。
**DenseNet这个名字由变量之间的“稠密连接”而得来，最后一层与之前的所有层紧密相连**。
稠密连接如图7.7.2所示。
<center>
    <img src="../img/densenet.svg" alt="稠密连接">
</center>
<center>图7.7.2 稠密连接</center><br>
稠密网络主要由2部分构成：<b>稠密块（dense block）</b> 和 <b>过渡层（transition layer）</b>。

前者定义如何连接输入和输出，而后者则控制通道数量，使其不会太复杂。

- **要点：**
  - **从ResNet到DenseNet**：ResNet通过恒等映射和跳跃连接实现信息的传递，将函数$f$分解为两部分：一个简单的线性项和一个复杂的非线性项。然而，如果我们希望将$f$拓展成超过两部分的信息，DenseNet提供了一种可能的方案。
  - **DenseNet的运行机制**：DenseNet的关键区别在于，它的输出是**连接**（在图中由$[,]$表示），而不是像ResNet那样的简单相加。这意味着，DenseNet执行了从$\mathbf{x}$到其泰勒展开式的映射，将卷积层的输出与X在维度上连接起来形成新的张量，而不是进行算术运算。
  - **DenseNet的命名**：DenseNet这个名字来源于变量之间的“稠密连接”，最后一层与之前的所有层紧密相连。
  - **DenseNet的组成**：DenseNet主要由两部分构成——**稠密块（dense block）**和**过渡层（transition layer）**。稠密块定义了如何连接输入和输出，而过渡层则**控制通道数量**，以防止模型过于复杂。

## 7.7.2 稠密块体
- DenseNet使用了ResNet改良版的“批量规范化、激活和卷积”架构（参见 7.6节中的练习）。
- 首先实现一下这个架构。

In [2]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l

def conv_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1))

- 一个**稠密块**由多个卷积块组成，每个卷积块使用相同数量的输出通道。
- 然而，在前向传播中，将每个卷积块的输入和输出**在通道维上连结**。


In [3]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(
                num_channels * i + input_channels, num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # 连接通道维度上每个块的输入和输出
            X = torch.cat((X, Y), dim=1)
        return X

- 在下面的例子中，定义一个有2个输出通道数为10的(**`DenseBlock`**)。
- 使用通道数为3的输入时，会得到通道数为$3+2\times 10=23$的输出。
- 卷积块的通道数控制了输出通道数相对于输入通道数的增长，因此也被称为**增长率（growth rate）**。


In [4]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

- **示例：在通道维连接张量**

In [5]:
## 在通道维即第2维连接张量,DenseNet的特定操作
a = torch.randn(960).reshape((5,3,8,8))
b = torch.randn(960).reshape((5,3,8,8))
a.shape,b.shape,torch.cat((a,b),dim=1).shape,torch.cat((a,b,a),dim=1).shape

(torch.Size([5, 3, 8, 8]),
 torch.Size([5, 3, 8, 8]),
 torch.Size([5, 6, 8, 8]),
 torch.Size([5, 9, 8, 8]))

## 7.7.3 过渡层

- 由于每个稠密块都会带来通道数的增加，使用过多则会**过于复杂化**模型。
- 过渡层可以用来控制模型复杂度。
  - 它通过$1\times 1$卷积层来减小通道数，并使用步幅为2的平均池化层**减半高和宽**，从而进一步降低模型复杂度。


In [6]:
def transition_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2))

- 对上一个例子中稠密块的输出使用通道数为10的过渡层。
- 此时输出的通道数减为10，高和宽均减半。


In [7]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

## 7.7.4 DenseNet模型

- 构造DenseNet模型。
- DenseNet首先使用同ResNet一样的单卷积层和最大池化层。


In [8]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

- 接下来，类似于ResNet使用的4个残差块，DenseNet使用的是4个稠密块。
- 与ResNet类似，我们可以设置每个稠密块使用多少个卷积层。
  - 这里设成4，从而与7.6节的ResNet-18保持一致。
  - 稠密块里的卷积层通道数（即增长率）设为32，所以每个稠密块将增加128个通道。
- 在每个模块之间，ResNet通过步幅为2的残差块减小高和宽，DenseNet则使用过渡层来减半高和宽，并减半通道数。


In [9]:
# num_channels为当前的通道数
num_channels, growth_rate = 64, 32 
num_convs_in_dense_blocks = [4, 4, 4, 4]
blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    # 上一个稠密块的输出通道数
    num_channels += num_convs * growth_rate
    # 在稠密块之间添加一个转换层，使通道数量减半
    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

- 与ResNet类似，最后接上全局池化层和全连接层来输出结果。


In [10]:
net = nn.Sequential(
    b1, *blks,
    nn.BatchNorm2d(num_channels), nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10))

## 7.7.5 训练模型
- 由于这里使用了比较深的网络，本节里将输入高和宽从224降到96来简化计算。


In [16]:
import time
import numpy as np
lr, num_epochs, batch_size = 0.1, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
start = time.time()
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
end = time.time()
print(f'3090ti上模型训练耗时：{int(np.floor((end-start)/60))}分钟{(int(end-start)%60) }秒')

<img src='..\img\7_7_1.png' height=350 width=350>

## 小结

* 在跨层连接上，不同于ResNet中将输入与输出相加，稠密连接网络（DenseNet）在通道维上连结输入与输出。
* DenseNet的主要构建模块是稠密块和过渡层。
* 在构建DenseNet时，我们需要通过添加过渡层来控制网络的维数，从而再次减少通道的数量。

- **说明：本节DenseNet的架构**
- 假设输入数据形状为(batch_size, 1, 96, 96)，那么下面的输出形状将如下所示：
  - Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)): 输出形状为(batch_size, 64, 48, 48)
  - BatchNorm2d(64): 输出形状为(batch_size, 64, 48, 48)
  - ReLU(): 输出形状为(batch_size, 64, 48, 48)
  - MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False): 输出形状为(batch_size, 64, 24, 24)
- 然后进入第一个DenseBlock，每个DenseBlock内部的操作都会增加通道数。例如在第一个DenseBlock中，每个Sequential里的Conv2d层都会增加32个通道，所以：
  - 第一个DenseBlock结束后：输出形状为(batch_size, 192, 24, 24)
- 接着是转换层（包括BatchNorm、ReLU、Conv2d和AvgPool2d）：
  - Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1)): 输出形状为(batch_size, 96, 24, 24)
  - AvgPool2d(kernel_size=2, stride=2, padding=0): 输出形状为(batch_size, 96, 12, 12)
- 同样地，经过第二个DenseBlock：
  - 第二个DenseBlock结束后：输出形状为(batch_size, 224, 12, 12)
- 再经过一个转换层：
  - Conv2d(224, 112, kernel_size=(1, 1), stride=(1, 1)): 输出形状为(batch_size, 112, 12, 12)
  - AvgPool2d(kernel_size=2, stride=2, padding=0): 输出形状为(batch_size, 112, 6, 6)
- 同样地，经过第三个DenseBlock：
  - 第三个DenseBlock结束后：输出形状为(batch_size, 240, 6, 6)
- 再经过一个转换层：
  - Conv2d(240, 120, kernel_size=(1, 1), stride=(1, 1)): 输出形状为(batch_size, 120, 6, 6)
  - AvgPool2d(kernel_size=2, stride=2, padding=0): 输出形状为(batch_size, 120, 3, 3)
- 最后，经过第四个DenseBlock：
  - 第四个DenseBlock结束后：输出形状为(batch_size, 248, 3, 3)
- 然后是BatchNorm2d(248)和ReLU()，它们不改变形状，保持为(batch_size, 248, 3, 3)。
- AdaptiveAvgPool2d(output_size=(1, 1))将特征图大小降为1x1，因此输出形状为(batch_size, 248, 1, 1)。
- Flatten(start_dim=1, end_dim=-1)将特征展平，输出形状为(batch_size, 248)。
- 最后，Linear(in_features=248, out_features=10, bias=True)将248维的输入映射到10个输出类别，因此最终输出形状为(batch_size, 10)。


- **（2）DenseNet的细节**

In [11]:
net

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): DenseBlock(
    (net): Sequential(
      (0): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): Sequential(
        (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (2): Sequential(
        (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1